In [2]:
import sqlite3
import pandas as pd
import sys
import numpy as np
import argparse
from os import listdir
from os.path import isfile, join

class args:
    i = "/Users/seunfuta/Downloads/NIST/IMG/sdelete-W7x64.db" #Wireshark-W7x64.db"
    c = "/Users/seunfuta/Downloads/NIST/OluDB_combo_v3.db"
    o = "/Users/seunfuta/Downloads/NIST/OLUSCAN/"
if __name__ == '__main__':
    #parser = argparse.ArgumentParser(description='Olu method')
    #parser.add_argument('-c', action="store", default="/Users/seunfuta/Downloads/NIST/OluDB_combo_v3.db", help='catalog path')
    #parser.add_argument('-i', action="store", default="/Users/seunfuta/Downloads/NIST/IMG/Wireshark-W7x64.db", help="image path folder")
    #parser.add_argument('-o', action="store", default="/Users/seunfuta/Downloads/NIST/OLUSCAN/", help='output csv')
    #args = parser.parse_args()
    
    #onlyfiles = [f for f in listdir(args.i) if isfile(join(args.i, f))]
    #for file in onlyfiles:
    #    print("IMAGE "+str(onlyfiles.index(file)) +" out of "+str(len(onlyfiles)))
    #    main(join(args.i+file))
    #main(join(args.i))
    #def main(imagefilepath):
    CATALOG_DB_PATH = args.c
    catalog_conn = sqlite3.connect(CATALOG_DB_PATH)
    catalog_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize','app','app_id'])
    catalog_df = pd.read_sql_query("SELECT block_hashes.obj_id, block_hashes.inode, block_hashes.filename, block_hashes.file_offset, \
                    block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize , files.app, files.app_id\
                    FROM files \
                    INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id \
                    and files.inode = block_hashes.inode and files.filename=block_hashes.filename;", catalog_conn)
    print("original length ",len(catalog_df))
    catalog_df = catalog_df[catalog_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
    catalog_df = catalog_df[catalog_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
    catalog_df = catalog_df[catalog_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
    catalog_df = catalog_df[catalog_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
    catalog_conn.close()
    print("catalog app length, ", len(catalog_df))
    #############
    print(args.i)#magefilepath)
    IMAGE_DB_PATH = args.i#imagefilepath
    image_conn = sqlite3.connect(IMAGE_DB_PATH)
    image_df = pd.DataFrame(columns=['obj_id', 'inode', 'filename','file_offset', 'len','md5','sha1', 'partition', 'filesize'])
    image_df = pd.read_sql_query("SELECT block_hashes.obj_id, files.inode, files.filename, block_hashes.file_offset, \
                    block_hashes.len, block_hashes.md5, block_hashes.sha1, files.partition,files.filesize \
                    FROM files \
                    INNER JOIN block_hashes ON files.obj_id = block_hashes.obj_id;", image_conn)
    print("original image length ",len(image_df))
    image_df = image_df[image_df.md5 != 'bf619eac0cdf3f68d496ea9344137e8b']
    image_df = image_df[image_df.md5 != 'de03fe65a6765caa8c91343acc62cffc']
    image_df = image_df[image_df.md5 != '85eba416ce0ee0951d1d93e73b191b75']
    image_df = image_df[image_df.md5 != '1b5c2cbf1e37f6b0d33751269ae707af']
    print("current image length ",len(image_df))
    ##############


original length  6826014
catalog app length,  6546710
/Users/seunfuta/Downloads/NIST/IMG/sdelete-W7x64.db
original image length  6415822
current image length  875101


In [3]:
    from collections import OrderedDict
    app_list = catalog_df.app.unique() #['Wireshark-W7x64'] #
    result_df = pd.DataFrame()
    for app in ['TrueCrypt63-WinXP']:#app_list:
        #if app == 'OfficePro2003-W7x32':
        app_df = catalog_df[catalog_df.app == app]
        app_unique_md5s= app_df.md5.unique()
        matched_image_df = image_df[image_df.md5.isin(app_unique_md5s)]
        #print(app," matched number of sectors in image ",len(matched_image_df))
        #print(matched_image_df)
        #lets create app sec pairs
        app_files = app_df.filename.unique()
        #print(app_files)
        app_pairs_set = {} #dict #set() 
        #lst2 = list(matched_image_df.md5)
        #lst2_pairs = list(map(lambda a, b: a + b, lst2[:-1], lst2[1:]))
        #Prob_Total = float(0)
        file1 = app_files[0]
        print(file1)



Program Files/TrueCrypt/TrueCrypt User Guide.pdf


In [4]:
        file1 = app_files[2]
        print(file1)

Program Files/TrueCrypt/TrueCrypt Format.exe


In [5]:
files_df = app_df[app_df.filename == file1]

In [7]:
files_df.reset_index(drop=True, inplace=True)
print(files_df)


      obj_id  inode                                      filename  \
0       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
1       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
2       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
4       1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
...      ...    ...                                           ...   
3031    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3032    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3033    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3034    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   
3035    1950  10929  Program Files/TrueCrypt/TrueCrypt Format.exe   

      file_offset  len                               md5  \
0               0  512  f7f9fd15dc6b25615dced7f50e57922c   
1             512  512  b640f487d6a0d49506266a74f88

In [37]:
file_hashes = files_df.md5.unique()
matched_image_df['image_offset'] = matched_image_df.index
matched_appfile_image_df = matched_image_df[matched_image_df.md5.isin(file_hashes)]
print("orig. image: ",len(matched_image_df), " new image: ", len(matched_appfile_image_df))

orig. image:  85  new image:  66


/var/folders/s7/d__51l693s13d_yt81j0qp9r0000gn/T/ipykernel_769/654800188.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_image_df['image_offset'] = matched_image_df.index


In [38]:
print(matched_appfile_image_df.md5)

656673     b27431292abccda8fec17eb690b82eb8
1067731    b27431292abccda8fec17eb690b82eb8
5716000    4f73573c1b597d9e6ab273854880b6c7
5716001    e557020f519052b3b696ace0d10d4fef
5716337    4f73573c1b597d9e6ab273854880b6c7
                         ...               
6301887    e557020f519052b3b696ace0d10d4fef
6301904    4f73573c1b597d9e6ab273854880b6c7
6301905    e557020f519052b3b696ace0d10d4fef
6302272    4f73573c1b597d9e6ab273854880b6c7
6302273    e557020f519052b3b696ace0d10d4fef
Name: md5, Length: 66, dtype: object


In [39]:
cat_off = np.array(files_df.index[files_df.md5 == 'b27431292abccda8fec17eb690b82eb8'])
cat_off = cat_off % 8
print(cat_off)

[0]


In [40]:
matched_off = np.array(matched_appfile_image_df.index[matched_appfile_image_df.md5 == 'b27431292abccda8fec17eb690b82eb8'])
print(matched_off)
matched_off = matched_off % 8
print(matched_off)
print(type(matched_off))

[ 656673 1067731]
[1 3]
<class 'numpy.ndarray'>


In [34]:
matched_off in cat_off
file_hashes_index_dict = {}

False

In [34]:
def condition(row):
    #print(row)
    matched_off = np.array(matched_appfile_image_df.index[matched_appfile_image_df.md5 == row])
    print(matched_off)
    matched_off = matched_off % 8
    print(matched_off)
    cat_off = np.array(files_df.index[files_df.md5 == row])
    print(cat_off)
    cat_off = cat_off % 8
    print(cat_off)
    return matched_off in cat_off

result = np.where(condition(matched_appfile_image_df.md5), 1, 0)

[ 656673 1067731 5716000 5716001 5716337 5716338 5716485 5716486 5716633
 5716634 5716790 5716791 5716950 5716951 5717107 5717108 5717265 5717266
 5717424 5717425 5717581 5717582 5717732 5717733 5717882 5717883 5718039
 5718040 5718195 5718196 5718353 5718354 5718510 5718511 5719444 5719445
 5719800 5719801 5721583 5721584 5722270 5722271 5723330 5723331 5723465
 5723466 6299562 6299563 6300531 6300844 6300845 6301642 6301643 6301689
 6301690 6301708 6301709 6301819 6301868 6301869 6301886 6301887 6301904
 6301905 6302272 6302273]
[1 3 0 1 1 2 5 6 1 2 6 7 6 7 3 4 1 2 0 1 5 6 4 5 2 3 7 0 3 4 1 2 6 7 4 5 0
 1 7 0 6 7 2 3 1 2 2 3 3 4 5 2 3 1 2 4 5 3 4 5 6 7 0 1 0 1]


ValueError: Can only compare identically-labeled Series objects

In [47]:
#from tqdm import tqdm
#import time
df_dict = matched_appfile_image_df.to_dict('records')
for row in df_dict:
    cluster_off = row['image_offset'] % 8
    catappfilehash_off = np.array(files_df.index[files_df.md5 == row['md5']])
    catappfilehash_off = catappfilehash_off % 8
    print(cluster_off, row['md5'],cluster_off in catappfilehash_off)

1 b27431292abccda8fec17eb690b82eb8 False
3 b27431292abccda8fec17eb690b82eb8 False
0 4f73573c1b597d9e6ab273854880b6c7 False
1 e557020f519052b3b696ace0d10d4fef False
1 4f73573c1b597d9e6ab273854880b6c7 False
2 e557020f519052b3b696ace0d10d4fef False
5 4f73573c1b597d9e6ab273854880b6c7 False
6 e557020f519052b3b696ace0d10d4fef False
1 4f73573c1b597d9e6ab273854880b6c7 False
2 e557020f519052b3b696ace0d10d4fef False
6 4f73573c1b597d9e6ab273854880b6c7 True
7 e557020f519052b3b696ace0d10d4fef True
6 4f73573c1b597d9e6ab273854880b6c7 True
7 e557020f519052b3b696ace0d10d4fef True
3 4f73573c1b597d9e6ab273854880b6c7 False
4 e557020f519052b3b696ace0d10d4fef False
1 4f73573c1b597d9e6ab273854880b6c7 False
2 e557020f519052b3b696ace0d10d4fef False
0 4f73573c1b597d9e6ab273854880b6c7 False
1 e557020f519052b3b696ace0d10d4fef False
5 4f73573c1b597d9e6ab273854880b6c7 False
6 e557020f519052b3b696ace0d10d4fef False
4 4f73573c1b597d9e6ab273854880b6c7 False
5 e557020f519052b3b696ace0d10d4fef False
2 4f73573c1b597d9e6a

In [26]:
files_hashes = files_df.md5.unique()


def is_clusteroff_valid(hash,ind):
    if true:
        return True




result = np.where(is_clusteroff_valid(matched_appfile_image_df,files_df), 1, 0)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
        for file in list(file1):#app_files:
            #file_hash_dict = {}
            #file_hashpair_set = set()
            files_df = app_df[app_df.filename == file]
            #print(f'file {file} is of size {len(files_df)}')
            file_hashes = files_df.md5
            '''
            if len(file_hashes)< 2:
                 #print(file_hashes.iloc[0])
                file_hashpair_set.add(file_hashes.iloc[0])
            else:
                for i in range(0, len(file_hashes)-1):
                    #print(file_hashes.iloc[i])
                    #print(file_hashes.iloc[int(i+1)])
                    hash_pair = file_hashes.iloc[i]+file_hashes.iloc[i+1]
                #print(i, hash_pair)
                    file_hashpair_set.add(hash_pair)
            app_pairs_set[file] = file_hashpair_set 
            '''
            file_hash_uniq_list = files_df.md5.unique()
            hash2pos = {}
            #result = hash2pos[]
            image_2_file_df = image_df[image_df.md5.isin(file_hash_uniq_list)]
            for uniq_hash in file_hash_uniq_list:
                hash2pos[uniq_hash] = list(map(lambda x: file_hash_uniq_list.index(x), uniq_hash))
            #image_hashes_dict = OrderedDict()
            result = list(map(lambda a, b, c: np.where((b-a ==1) & (c-b==1),1,0), hash2pos[image_2_file_df[:-2]], hash2pos[image_2_file_df[1:-1]], hash2pos[image_2_file_df[2:]]))
            